<a href="https://colab.research.google.com/github/prathmeshtiwari22/Pract/blob/main/Bigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [11]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', "'", ',', '-', '.', '1', '3', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [12]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [13]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[59, 65,  1, 65, 60,  1, 64, 65],
        [64, 54, 65, 50, 49,  1, 47, 50],
        [63, 50,  1, 65, 53, 46, 65,  1],
        [50,  1, 48, 46, 63, 50, 51, 66]])
targets:
tensor([[65,  1, 65, 60,  1, 64, 65, 46],
        [54, 65, 50, 49,  1, 47, 50, 64],
        [50,  1, 65, 53, 46, 65,  1, 46],
        [ 1, 48, 46, 63, 50, 51, 66, 57]])


In [14]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


LYqysYw7z6FY,kEGf
je-sFKCRjRzOot;QBjHgDz-oqER-sRVSB5-svWRDYT[A59xV  ZxBBug6.VsF!7AGVfjpoa[pt]7kw5ZB7nPYOa?,hWSSiMb1Y 87sH
jR﻿.9?:Nwn].kg eORDg﻿JWUOEKMDAwT[povJA5xr_kz:9K: 1BN'.F8nJCyK;LhB-fn6iDeMq?mKhY-pn3lrwR﻿R3wxoIaSLF﻿lwqJ qJO51BG.TNiOxf_hCn3l QSo9a1B6!yKR
G,h9z,h"gw8F]Uh1B,Vi8F﻿FcgwD8seP_39bUE;Lf_m
5[.A.7hG_H,zPA6y5U!1Bx
[YrGV8E"dDj.Aw]sWKYYkw px
HWB-OKBqqPeMd.LGTox!D;LhG:,vuw[hYdk.dwT1B[hV﻿otyJO"[UpI:tLvWT7hEj]7"y'av8Q.5:!WOm7t]spVPL?PldK!DDIzHY?﻿j?﻿fGuklAhDAh1QVH1BhxilFYYzOw; ﻿ldIilM6p5gLf


In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.743, val loss: 4.726
step: 250, train loss: 4.659, val loss: 4.669
step: 500, train loss: 4.601, val loss: 4.614
step: 750, train loss: 4.553, val loss: 4.542
4.391861915588379


Loss function = How many mistakes you made.

Optimizer = Teacher correcting you step by step.

GD = slow careful walk.

SGD = quick, noisy steps.

Momentum = adds speed + direction.

RMSprop = adjusts step size smartly.

Adam = best of Momentum + RMSprop.

AdamW = Adam + discipline (regularization).

In [17]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


jE yeTo!CL3 eMvPyUpshJE'yjwFj.!'ZI"]6﻿NiM-jE﻿PqfzOSii:xdk:c]CV SM6_yUzIu'f1u1YyNiFpiC!GWbF,rVJ?:,eKlM15qu1' LwM3"OS] B8m rCE]qsm?eMvg9zfqFl;g6 v;﻿jc1j _I-a7UA
j?u66B[yQNPJAs_W1BId?PU5WWOiJ7bVs;cA'Yqm,nJT]7buUn]W9 C]U3bh96Q]E:FvDldU7sc,hVa PIKbWW,dN'ZrLGs!htV]7am nA5ONilsR6xou].!wtVH﻿sWD":P1BR:9z-T"c_pf953sp5[; ldJ15wnJd
iyOw;L87!n:A.AsLSrJNmn??JSM!9zG5]A';xffAaSRe-1oG,jK3,Ply-sY_u6KDlFh1BN7bd1Btv]su9K_f!ks3vai!WMqPN]sRCUskvglWrZYMgaCciOxSZ'oQ1B8bd,VPaJ -ata]:'zFiRV":6pTEAu'.k8ZtzTfldC6 T]3caOk9h
